## 0. List all logs

In [1]:
import os
import csv
from typing import List, Dict, Tuple, Optional

import os

folder_path = './benchmark/contracts/34'

sol_files_info = []

for root, dirs, files in os.walk(folder_path):
    for file in files:
        if file.endswith('.sol'):
            file_path = os.path.join(root, file)
            with open(file_path, 'r', encoding='utf-8') as f:
                file_content = f.read()
            sol_files_info.append({
                'name': file,
                'path': file_path
            })
print(f"Number of files in the folder: : {len(sol_files_info)}")
print(f"List all files in the folder {sol_files_info}")

Number of files in the folder: : 61
List all files in the folder [{'name': 'PrizeFlush.sol', 'path': './benchmark/contracts/34/v4-periphery/contracts/PrizeFlush.sol'}, {'name': 'IPrizeFlush.sol', 'path': './benchmark/contracts/34/v4-periphery/contracts/interfaces/IPrizeFlush.sol'}, {'name': 'IStrategy.sol', 'path': './benchmark/contracts/34/v4-periphery/contracts/interfaces/IStrategy.sol'}, {'name': 'IReserve.sol', 'path': './benchmark/contracts/34/v4-periphery/contracts/interfaces/IReserve.sol'}, {'name': 'DrawCalculatorTimelock.sol', 'path': './benchmark/contracts/34/v4-timelocks/contracts/DrawCalculatorTimelock.sol'}, {'name': 'L2TimelockTrigger.sol', 'path': './benchmark/contracts/34/v4-timelocks/contracts/L2TimelockTrigger.sol'}, {'name': 'L1TimelockTrigger.sol', 'path': './benchmark/contracts/34/v4-timelocks/contracts/L1TimelockTrigger.sol'}, {'name': 'IDrawCalculatorTimelock.sol', 'path': './benchmark/contracts/34/v4-timelocks/contracts/interfaces/IDrawCalculatorTimelock.sol'}, 

## 1. TA Model Results Process

## 1.1 Extract all results in Log file

In this process, we provide all functions that can extract the vulnerability types and number from the log file and store them in a list of Vulnerability objects.

In [2]:
import re
from collections import defaultdict

class Vulnerability:
    def __init__(self, type, line_number):
        self.type = type
        self.line_number = line_number

def extract_vulnerabilities(log_file_path):
    vulnerability_patterns = [
        r"<INFO> Integer Overflow/Underflow Identified",
        r"<INFO> Reentrancy Identified",
        r"<INFO> Unchecked External Calls Identified",
        r"<INFO> Unchecked Low-level Calls Identified",
        r"<INFO> Transactions Order Dependence Identified",
        r"<INFO> Timestamp Dependency Identified",
        r"<INFO> Predictable Randomness Identified",
        r"<INFO> Unsafe `tx.origin` Identified",
        r"<INFO> Unsafe Suicide Identified",
        r"<INFO> Gas Limit Identified",
        r"<INFO> Price Manipulation Identified",
        r"<INFO> Assumption of Fixed-Length Array Identified",
        r"<INFO> Lack of Withdrawal Function Identified",
        r"<INFO> Lack of Authorization Identified",
        r"<INFO> Potential Data Inconsistency Identified",
        r"<INFO> Hash Collision Identified",
        r"<INFO> Uninitialized Return Variable Identified",
        r"<INFO> Misdeclared Constructor Identified",
        r"<INFO> Missing 'onlyOwner' Modifier Identified",
        r"<INFO> Misuse of `msg.value` within a Loop Identified",
        r"<INFO> Precision Loss Identified",
        r"<INFO> Redundant Conditional Check Identified",
        r"<INFO> External Oracle Dependency Identified",
        r"<INFO> Ownership Hijacking Identified",
        r"<INFO> Centralization Risk Identified", 
        r"<INFO> Incorrect Funding Rate Calculation Identified",
        r"<INFO> Flash Loan Fee Manipulation Identified",
        r"<INFO> Misuse of Mapping Getter Identified",
        r"<INFO> Lack of Getter Function Implementation Identified",
        r"<INFO> Unnecessary Comparison Identified",
        r"<INFO> Inconsistent Initialization Identified",
        r"<INFO> Potential Inconsistency in Yield Source Swapping Identified",
        r"<INFO> Incorrect Signer Verification Identified",
        r"<INFO> Order of Inheritance Initialization Identified",
        r"<INFO> Impracticality of Exact Match Identified",
        r"<INFO> Inconsistent Base Tokens Identified",
        r"<INFO> Handling Partial Withdrawals Identified",
        r"<INFO> Fallback Function Identified",
        r"<INFO> Unlimited Token Approval Identified",
        r"<INFO> Lack of Input Validation Identified",
        r"<INFO> DoS Vulnerability Identified"   
    ]
        
    vulnerability_occurrences = defaultdict(list)
    
    try:
        with open(log_file_path, 'r', encoding='utf-8') as file:
            for line_number, line in enumerate(file, 1):
                for pattern in vulnerability_patterns:
                    if re.search(pattern, line):
                        vulnerability_occurrences[pattern].append(line_number)
                        # print(f"DEBUG: Found '{pattern}' at line {line_number}")
    except FileNotFoundError:
        print(f"Error: The file '{log_file_path}' was not found.")
        return []
    except IOError as e:
        print(f"Error reading the file: {e}")
        return []
    
    vulnerability_list = []
    for pattern, line_numbers in vulnerability_occurrences.items():
        if len(line_numbers) >= 2:
            vulnerability = Vulnerability(pattern, line_numbers)
            vulnerability_list.append(vulnerability)
    
    print(f"DEBUG: Total vulnerabilities found: {len(vulnerability_list)}")
    return vulnerability_list

def clean_file_name(file_name: str) -> str:
    """
    Remove the 'ArSC_TA_' prefix from the file name.

    Args:
    file_name (str): Original file name.

    Returns:
    str: Cleaned file name.
    """
    return re.sub(r'^ArSC_TA_', '', file_name)

def process_log_folder(folder_path: str) -> List[Tuple[str, int, str]]:
    """
    Process all log files in the given folder and extract vulnerabilities.

    Args:
    folder_path (str): Path to the folder containing log files.

    Returns:
    List[Tuple[str, int, str]]: List of tuples containing (file name, number of occurrences, vulnerability type).
    """
    results: List[Tuple[str, int, str]] = []
    
    for filename in os.listdir(folder_path):
        if filename.endswith('.log'):
            file_path = os.path.join(folder_path, filename)
            vulnerabilities = extract_vulnerabilities(file_path)
            # print(f"DEBUG: Found {len(vulnerabilities)} vulnerabilities in file: {filename}")
            
            # cleaned_filename = clean_file_name(filename)
            # print(f"DEBUG: Processing file: {cleaned_filename}")
            # for vuln in vulnerabilities:
            #     results.append((cleaned_filename, len(vulnerabilities), vuln.type))
            
            if vulnerabilities:
                cleaned_filename = clean_file_name(filename)
                total_occurrences = len(vulnerabilities)
                combined_types = ', '.join(sorted(set(vuln.type for vuln in vulnerabilities)))
                results.append((cleaned_filename, total_occurrences, combined_types))
    
    return results

def save_to_csv(results: List[Tuple[str, int, str]], output_file: str) -> None:
    """
    Save the vulnerability results to a CSV file.

    Args:
    results (List[Tuple[str, int, str]]): List of vulnerability results.
    output_file (str): Path to the output CSV file.
    """
    with open(output_file, 'w', newline='', encoding='utf-8') as csvfile:
        csv_writer = csv.writer(csvfile)
        csv_writer.writerow(['File Name', 'Number', 'Vulnerability Type', 'All Details'])
        csv_writer.writerows(results)

## 1.2 Reports
### 1.2.1 Overall Evaluation Reports 
Generate the overall evaluation report for all the log files in the folder. The report should include the following information:
- The number of vulnerabilities found in each log file.
- The type of vulnerability found (e.g., SQL injection, XSS, etc.).
- The line numbers where the vulnerability was found.
- The content of the log file where the vulnerability was found.
- The total number of vulnerabilities found across all log files.

In [39]:
log_folder_path = "/Users/weizhiyuan/Documents/code/SC_LLM_ChatDev/evaluationResult/TA_logs_extral"  # Replace with the actual path to your log file
output_csv_file = "/Users/weizhiyuan/Documents/code/SC_LLM_ChatDev/TA_vulnerability_results_1.csv"

print(f"Processing log files in folder: {log_folder_path}")

results = process_log_folder(log_folder_path)

if not results:
    print("No vulnerabilities found or there was an error processing the files.")

save_to_csv(results, output_csv_file)

print(f"\nResults saved to {output_csv_file}")
print(f"Total vulnerabilities found: {len(results)}")

Processing log files in folder: /Users/weizhiyuan/Documents/code/SC_LLM_ChatDev/TA_logs
DEBUG: Total vulnerabilities found: 12
DEBUG: Total vulnerabilities found: 12
DEBUG: Total vulnerabilities found: 13
DEBUG: Total vulnerabilities found: 6
DEBUG: Total vulnerabilities found: 10
DEBUG: Total vulnerabilities found: 9
DEBUG: Total vulnerabilities found: 7
DEBUG: Total vulnerabilities found: 5
DEBUG: Total vulnerabilities found: 10
DEBUG: Total vulnerabilities found: 10
DEBUG: Total vulnerabilities found: 9
DEBUG: Total vulnerabilities found: 9
DEBUG: Total vulnerabilities found: 11
DEBUG: Total vulnerabilities found: 12
DEBUG: Total vulnerabilities found: 14
DEBUG: Total vulnerabilities found: 11
DEBUG: Total vulnerabilities found: 11
DEBUG: Total vulnerabilities found: 7
DEBUG: Total vulnerabilities found: 7
DEBUG: Total vulnerabilities found: 12
DEBUG: Total vulnerabilities found: 13
DEBUG: Total vulnerabilities found: 10
DEBUG: Total vulnerabilities found: 0
DEBUG: Total vulnerabili

### 1.2.2 Extract Each log content in corresponding CSV file

we extract all certain vulnerability types (whether it exists)

In [3]:
import os
import re
import pandas as pd

# Function to extract sections based on specified tags
def extract_sections(log_content, tags):
    extracted_data = {}
    for tag in tags:
        pattern = re.compile(rf'<INFO> {tag}.*?(?=<INFO>|$)', re.DOTALL)
        matches = pattern.findall(log_content)
        extracted_data[tag] = matches
    return extracted_data


# Function to process a single log file
def process_log_file(file_path, tags, output_folder):
    # Read the log file
    with open(file_path, 'r') as file:
        log_content = file.read()

    # Extract sections
    extracted_data = extract_sections(log_content, tags)

    # Convert the extracted data to a DataFrame
    extracted_data_df = pd.DataFrame.from_dict(extracted_data, orient='index').transpose()

    # Generate the output file path
    log_file_name = os.path.splitext(os.path.basename(file_path))[0]
    output_file_path = os.path.join(output_folder, f"{log_file_name}_extracted.csv")

    # Save the DataFrame to a CSV file
    extracted_data_df.to_csv(output_file_path, index=False)

    return extracted_data_df

# Function to process all log files in a folder
def process_log_files_in_folder(folder_path, tags, output_folder):
    log_files = [f for f in os.listdir(folder_path) if os.path.isfile(os.path.join(folder_path, f)) and f.endswith('.log')]
    all_extracted_data = {}
    
    for log_file in log_files:
        print(log_file)
        file_path = os.path.join(folder_path, log_file)
        extracted_data_df = process_log_file(file_path, tags, output_folder)
        all_extracted_data[log_file] = extracted_data_df
    
    return all_extracted_data

input_folder = '/Users/weizhiyuan/Documents/code/SC_LLM_ChatDev/evaluationResult/TA_logs_extral'  # Replace with your input folder path
output_folder = '/Users/weizhiyuan/Documents/code/SC_LLM_ChatDev/evaluationResult/TA_logs_extral/results'  # Replace with your output folder path

# Ensure the output folder exists
os.makedirs(output_folder, exist_ok=True)

# Tags to search for
tags = [
        "Integer Overflow/Underflow Identified",
        "Reentrancy Identified",
        "Unchecked External Calls Identified",
        "Unchecked Low-level Calls Identified",
        "Transactions Order Dependence Identified",
        "Timestamp Manipulation Identified",
        "Predictable Randomness Identified",
        "Unsafe `tx.origin` Identified",
        "Unsafe Suicide Identified",
        "Gas Limit Identified",
        "Price Manipulation Identified",
        "Assumption of Fixed-Length Array Identified",
        "Lack of Withdrawal Function Identified",
        "Lack of Authorization Identified",
        "Potential Data Inconsistency Identified",
        "Hash Collision Identified",
        "Uninitialized Return Variable Identified",
        "Misdeclared Constructor Identified",
        "Missing 'onlyOwner' Modifier Identified",
        "Misuse of `msg.value` within a Loop Identified",
        "Precision Loss Identified",
        "Redundant Conditional Check Identified",
        "External Oracle Dependency Identified",
        "Ownership Hijacking Identified",
        "Centralization Risk Identified", 
        "Incorrect Funding Rate Calculation Identified",
        "Flash Loan Fee Manipulation Identified",
        "Misuse of Mapping Getter Identified",
        "Lack of Getter Function Implementation Identified",
        "Unnecessary Comparison Identified",
        "Inconsistent Initialization Identified",
        "Potential Inconsistency in Yield Source Swapping Identified",
        "Incorrect Signer Verification Identified",
        "Order of Inheritance Initialization Identified",
        "Impracticality of Exact Match Identified",
        "Inconsistent Base Tokens Identified",
        "Handling Partial Withdrawals Identified",
        "Fallback Function Identified",
        "Unlimited Token Approval Identified",
        "Lack of Input Validation Identified",
        "DoS Vulnerability Identified",   
]

# Process all log files in the folder
all_extracted_data = process_log_files_in_folder(input_folder, tags, output_folder)


ArSC_TA_Extral_12_Cauldron_AIFar_20240728094305.log
ArSC_TA_Extral_69_NFTXSimpleFeeDistributor_AIFar_20240728092503.log
ArSC_TA_Extral_106_NFTLoanFacilitator_AIFar_20240728090800.log
ArSC_TA_Extral_78_FlanBackstop_AIFar_20240728090153.log
ArSC_TA_Extral_38_Identity_AIFar_20240728094614.log
ArSC_TA_Extral_14_BadgerYieldSource_AIFar_20240728093334.log
ArSC_TA_Extral_13_RCFactory_AIFar_20240728092923.log
ArSC_TA_Extral_97_LiquidityPool_AIFar_20240728084431.log
ArSC_TA_Extral_97_LiquidityFarming_AIFar_20240728084749.log
ArSC_TA_Extral_30_Controller_AIFar_20240728093634.log
ArSC_TA_Extral_55_MapleLoan_AIFar_20240728084611.log
ArSC_TA_Extral_94_NFTMarketCreators_AIFar_20240728083826.log
ArSC_TA_Extral_8_NFTXVaultUpgradeable_AIFar_20240728085426.log
ArSC_TA_Extral_8_NFTXFeeDistributor_AIFar_20240728084530.log
ArSC_TA_Extral_68_SingleTokenJoinV2_AIFar_20240728091338.log
ArSC_TA_Extral_5_Pools_AIFar_20240728085940.log
ArSC_TA_Extral_14_IdleYieldSource_AIFar_20240728090439.log
ArSC_TA_Extral_193

## 2. BA Mode Result Process

In [7]:
import re
def BA_clean_file_name(file_name: str) -> str:
    """
    Remove the 'ArSC_TA_' prefix from the file name.

    Args:
    file_name (str): Original file name.

    Returns:
    str: Cleaned file name.
    """
    return re.sub(r'^ArSC_BA_', '', file_name)

def extract_seminar_conclusion(log_file_path: str) -> Optional[str]:
    """
    Extract the seminar conclusion from a log file.

    Args:
    log_file_path (str): Path to the log file.

    Returns:
    Optional[str]: The extracted seminar conclusion or None if not found.
    """
    conclusions = []
    try:
        with open(log_file_path, 'r', encoding='utf-8') as file:
            content = file.read()
            matches = re.findall(r'\*\*\[Seminar Conclusion\]\*\*(.*?)(?=\[2024-|\Z)', content, re.DOTALL)
            conclusions = [match.strip() for match in matches if match.strip()]
    except FileNotFoundError:
        print(f"Error: The file '{log_file_path}' was not found.")
    except IOError as e:
        print(f"Error reading the file: {e}")
    
    return conclusions[7] if len(conclusions) >= 8 else None


def BA_process_log_folder(folder_path: str) -> List[Tuple[str, Optional[str]]]:
    """
    Process all log files in the given folder and extract vulnerabilities and seminar conclusions.

    Args:
    folder_path (str): Path to the folder containing log files.

    Returns:
    List[str, Optional[str]]]: List of tuples containing (cleaned file name, seminar conclusion).
    """
    results: List[Tuple[str, Optional[str]]] = []
    
    for filename in os.listdir(folder_path):
        if filename.endswith('.log'):
            file_path = os.path.join(folder_path, filename)
            cleaned_filename = BA_clean_file_name(filename)
            seminar_conclusion = extract_seminar_conclusion(file_path)
            
            results.append((cleaned_filename, seminar_conclusion))
    
    return results

def BA_save_to_csv(results: List[Tuple[str, Optional[str]]], output_file: str) -> None:
    """
    Save the vulnerability results and seminar conclusions to a CSV file.

    Args:
    results (List[Tuple[str, int, str, Optional[str]]]): List of vulnerability results and seminar conclusions.
    output_file (str): Path to the output CSV file.
    """
    with open(output_file, 'w', newline='', encoding='utf-8') as csvfile:
        csv_writer = csv.writer(csvfile)
        csv_writer.writerow(['File Name', 'Seminar Conclusion'])
        csv_writer.writerows(results)

In [8]:
log_folder_path: str = "/Users/weizhiyuan/Documents/code/SC_LLM_ChatDev/BA_logs"  # Replace with the actual path to your log folder
output_csv_file: str = "BA_vulnerability_results.csv"  # Name of the output CSV file

print(f"Processing log files in folder: {log_folder_path}")

results = BA_process_log_folder(log_folder_path)

if not results:
    print("No vulnerabilities found or there was an error processing the files.")
else:
    BA_save_to_csv(results, output_csv_file)

    print(f"\nResults saved to {output_csv_file}")
    print(f"Total files processed: {len(results)}")     

Processing log files in folder: /Users/weizhiyuan/Documents/code/SC_LLM_ChatDev/BA_logs

Results saved to BA_vulnerability_results.csv
Total files processed: 101


## 3. Process log files in the folder and save the results to a CSV file

### 3.1 TA Processing Files

#### 3.1.1 Extract all results from log files

In [ ]:
import re
from collections import defaultdict

class Vulnerability:
    def __init__(self, type, line_number):
        self.type = type
        self.line_number = line_number

def extract_vulnerabilities(log_file_path):
    vulnerability_patterns = [
        r"<INFO> Integer Overflow/Underflow Identified",
        r"<INFO> Reentrancy Identified",
        r"<INFO> Unchecked External Calls Identified",
        r"<INFO> Unchecked Low-level Calls Identified",
        r"<INFO> Transactions Order Dependence Identified",
        r"<INFO> Timestamp Dependency Identified",
        r"<INFO> Predictable Randomness Identified",
        r"<INFO> Unsafe `tx.origin` Identified",
        r"<INFO> Unsafe Suicide Identified",
        r"<INFO> Gas Limit Identified",
        r"<INFO> Price Manipulation Identified",
        r"<INFO> Assumption of Fixed-Length Array Identified",
        r"<INFO> Lack of Withdrawal Function Identified",
        r"<INFO> Lack of Authorization Identified",
        r"<INFO> Potential Data Inconsistency Identified",
        r"<INFO> Hash Collision Identified",
        r"<INFO> Uninitialized Return Variable Identified",
        r"<INFO> Misdeclared Constructor Identified",
        r"<INFO> Missing 'onlyOwner' Modifier Identified",
        r"<INFO> Misuse of `msg.value` within a Loop Identified",
        r"<INFO> Precision Loss Identified",
        r"<INFO> Redundant Conditional Check Identified",
        r"<INFO> External Oracle Dependency Identified",
        r"<INFO> Ownership Hijacking Identified",
        r"<INFO> Centralization Risk Identified", 
        r"<INFO> Incorrect Funding Rate Calculation Identified",
        r"<INFO> Flash Loan Fee Manipulation Identified",
        r"<INFO> Misuse of Mapping Getter Identified",
        r"<INFO> Lack of Getter Function Implementation Identified",
        r"<INFO> Unnecessary Comparison Identified",
        r"<INFO> Inconsistent Initialization Identified",
        r"<INFO> Potential Inconsistency in Yield Source Swapping Identified",
        r"<INFO> Incorrect Signer Verification Identified",
        r"<INFO> Order of Inheritance Initialization Identified",
        r"<INFO> Impracticality of Exact Match Identified",
        r"<INFO> Inconsistent Base Tokens Identified",
        r"<INFO> Handling Partial Withdrawals Identified",
        r"<INFO> Fallback Function Identified",
        r"<INFO> Unlimited Token Approval Identified",
        r"<INFO> Lack of Input Validation Identified",
        r"<INFO> DoS Vulnerability Identified"   
    ]
        
    vulnerability_occurrences = defaultdict(list)
    
    try:
        with open(log_file_path, 'r', encoding='utf-8') as file:
            for line_number, line in enumerate(file, 1):
                for pattern in vulnerability_patterns:
                    if re.search(pattern, line):
                        vulnerability_occurrences[pattern].append(line_number)
                        # print(f"DEBUG: Found '{pattern}' at line {line_number}")
    except FileNotFoundError:
        print(f"Error: The file '{log_file_path}' was not found.")
        return []
    except IOError as e:
        print(f"Error reading the file: {e}")
        return []
    
    vulnerability_list = []
    for pattern, line_numbers in vulnerability_occurrences.items():
        if len(line_numbers) >= 2:
            vulnerability = Vulnerability(pattern, line_numbers)
            vulnerability_list.append(vulnerability)
    
    print(f"DEBUG: Total vulnerabilities found: {len(vulnerability_list)}")
    return vulnerability_list

def clean_file_name(file_name: str) -> str:
    """
    Remove the 'RealWorld_TA_' prefix from the file name.

    Args:
    file_name (str): Original file name.

    Returns:
    str: Cleaned file name.
    """
    return re.sub(r'^RealWorld_TA_', '', file_name)

def process_log_folder(folder_path: str) -> List[Tuple[str, int, str]]:
    """
    Process all log files in the given folder and extract vulnerabilities.

    Args:
    folder_path (str): Path to the folder containing log files.

    Returns:
    List[Tuple[str, int, str]]: List of tuples containing (file name, number of occurrences, vulnerability type).
    """
    results: List[Tuple[str, int, str]] = []
    
    for filename in os.listdir(folder_path):
        if filename.endswith('.log'):
            file_path = os.path.join(folder_path, filename)
            vulnerabilities = extract_vulnerabilities(file_path)
            # print(f"DEBUG: Found {len(vulnerabilities)} vulnerabilities in file: {filename}")
            
            # cleaned_filename = clean_file_name(filename)
            # print(f"DEBUG: Processing file: {cleaned_filename}")
            # for vuln in vulnerabilities:
            #     results.append((cleaned_filename, len(vulnerabilities), vuln.type))
            
            if vulnerabilities:
                cleaned_filename = clean_file_name(filename)
                total_occurrences = len(vulnerabilities)
                combined_types = ', '.join(sorted(set(vuln.type for vuln in vulnerabilities)))
                results.append((cleaned_filename, total_occurrences, combined_types))
    
    return results

def save_to_csv(results: List[Tuple[str, int, str]], output_file: str) -> None:
    """
    Save the vulnerability results to a CSV file.

    Args:
    results (List[Tuple[str, int, str]]): List of vulnerability results.
    output_file (str): Path to the output CSV file.
    """
    with open(output_file, 'w', newline='', encoding='utf-8') as csvfile:
        csv_writer = csv.writer(csvfile)
        csv_writer.writerow(['File Name', 'Number', 'Vulnerability Type', 'All Details'])
        csv_writer.writerows(results)

#### 3.1.2 Overall Evaluation Reports 
Generate the overall evaluation report for all the log files in the folder. The report should include the following information:
- The number of vulnerabilities found in each log file.
- The type of vulnerability found (e.g., SQL injection, XSS, etc.).
- The line numbers where the vulnerability was found.
- The content of the log file where the vulnerability was found.
- The total number of vulnerabilities found across all log files.

In [ ]:
log_folder_path = "./WD_TA/WD_TA_C8"  # Replace with the actual path to your log file
output_csv_file = "./WD_TA/WD_TA_C8_result.csv"

print(f"Processing log files in folder: {log_folder_path}")

results = process_log_folder(log_folder_path)

if not results:
    print("No vulnerabilities found or there was an error processing the files.")

save_to_csv(results, output_csv_file)

print(f"\nResults saved to {output_csv_file}")

### 3.2 BA Processing Files

In [ ]:
import os
import shutil

def find_and_copy_log_files(source_folder, destination_folder):
    log_files_info = []

    # Find all .log files
    for root, dirs, files in os.walk(source_folder):
        for file in files:
            if file.endswith('.log'):
                file_path = os.path.join(root, file)
                log_files_info.append({
                    'name': file,
                    'path': file_path
                })

    if not log_files_info:
        print("No .log files found in the source folder or its subfolders.")
        return

    # Create destination folder if it doesn't exist
    os.makedirs(destination_folder, exist_ok=True)

    # Copy files
    copied_files = 0
    for file_info in log_files_info:
        source_path = file_info['path']
        destination_path = os.path.join(destination_folder, file_info['name'])

        try:
            shutil.copy2(source_path, destination_path)
            print(f"Copied: {file_info['name']}")
            copied_files += 1
        except Exception as e:
            print(f"Error copying {file_info['name']}: {e}")

    print(f"\nTotal .log files found: {len(log_files_info)}")
    print(f"Total .log files copied: {copied_files}")

# Example usage
source_folder = './WD_BA_8'
destination_folder = './WD_BA_C8'

find_and_copy_log_files(source_folder, destination_folder)

In [ ]:
log_folder_path: str = "./WD_BA_C8/"  # Replace with the actual path to your log folder
output_csv_file: str = "WD_BA_C8_result.csv"  # Name of the output CSV file

print(f"Processing log files in folder: {log_folder_path}")

results = BA_process_log_folder(log_folder_path)

if not results:
    print("No vulnerabilities found or there was an error processing the files.")
else:
    BA_save_to_csv(results, output_csv_file)

    print(f"\nResults saved to {output_csv_file}")
    print(f"Total files processed: {len(results)}")     